# [MICRO-452:] Project Report



<p><b>Authors:</b> &nbsp;&emsp;&emsp;&emsp;&emsp;&emsp;Antoine Rol, Ruben Jungius, Hugo Masson, Jadd<br>
<b>Supervisors:</b> &nbsp;&emsp;&emsp;&emsp;Prof. Francesco Mondada<br>
<b>Due date:</b>  &nbsp;&nbsp;&nbsp;&emsp;&emsp;&emsp;&emsp;December 7th, 2023</p>
<b>Presentation date:</b> &nbsp;&nbsp;December 14th, 2023</p>

<h1><center> Thymhiker  </center></h1>
<img src="" style="width: 600px;"> 

<h1>Table of Contents<span class="tocSkip"></span></h1><br>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1.&nbsp;&nbsp;</span>Introduction</a></span></li></ul><ul class="toc-item"><li><a href="#Project-summary-,-hardware-and-choices" data-toc-modified-id="Project-summary-,-hardware-and-choices-2"><span class="toc-item-num">2.&nbsp;&nbsp;</span>Project summary, hardware and choices</a></li></ul><ul class="toc-item"><li><span><a href="#Vision" data-toc-modified-id="Vision-3"><span class="toc-item-num">3.&nbsp;&nbsp;</span>Vision</a></span></li></ul><ul class="toc-item"><li><a href="#Global-navigation" data-toc-modified-id="Global-navigation-4"><span class="toc-item-num">4.&nbsp;&nbsp;</span>Global navigation</a></li></ul><ul class="toc-item"><li><a href="#Local-navigation" data-toc-modified-id="Local-navigation-5"><span class="toc-item-num">5.&nbsp;&nbsp;</span>Local navigation</a></li></ul><ul class="toc-item"><li><a href="#Filtering" data-toc-modified-id="Filtering-6"><span class="toc-item-num">6.&nbsp;&nbsp;</span>Filtering</a></li></ul><ul class="toc-item"><li><a href="#Motion-control" data-toc-modified-id="Motion-control-7"><span class="toc-item-num">7.&nbsp;&nbsp;</span>Motion control</a></li></ul><ul class="toc-item"><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-8"><span class="toc-item-num">8.&nbsp;&nbsp;</span>Conclusion</a></span></li></ul><ul class="toc-item"><li><a href="#Main" data-toc-modified-id="Main-9"><span class="toc-item-num">9.&nbsp;&nbsp;</span>Main</a></ul> 

# I. Introduction
<a class="anchor" id="Introduction"></a>
<p style='text-align: justify;'> 

The narrative behind this mobile robotic project involves a hiker. He has to reach a goal position, which is a refuge defined by a tag. On his way, there are black obstacles, which are volcanoes that the Thymio should avoid. Fortunately, he has a plan where the volcanoes are marked. He just has to decide on the best path to stay safe, then proceed along this route and not forget to dodge rocks that are not marked on the map.
This project contains 5 majors parts. 

- The **vision** : The goal of this part is to obtain the obstacle map and the goal position in offline mode, and then acquire the Thymio's real-time position and orientation.

- **Global Path** : Utilizing the obstacle map from the Vision part, this section computes the optimal path for the hiker to stay safe. A visibility graph algorithm will be applied, resulting in a list of path point coordinates.

- **Local Naviguation** : This section prevents the hiker from colliding with rocks not on the obstacle map. A neural network will be used for this function

- **Motion control** : Using the list of path point coordinates from the Global Navigation part, this part controls the speed of the left and right motors to accurately follow the path.

- **Filtering** with Kalman filter : In this section, a Kalman filter will be implemented to achieve a more precise position estimation and orientation. The objective is to maintain reliable estimates of orientation and position even in the absence of the camera signal.

Below, you will find a grafcet showing the implementation of each part and their interconnections.

<img src="ImgRapport\grafcet.PNG" style="width: 700px;">


# II. Project summary, hardware and choices
<a class="anchor" id="Project-summary-,-hardware-and-choices"></a>
### Project summary

# III. Vision
<a class="anchor" id="Vision"></a>

### Introduction

The vision component of our project plays a crucial role in detecting the region of interest (ROI) and end-point, facilitating the seamless interaction between the Thymio robot and its environment. In this section, we detail the vision pipeline, encompassing Aruco marker detection, perspective transformation, and obstacle detection.

### Aruco Marker Creation
Before delving into the vision pipeline, we commence by designing and crafting markers that will play a pivotal role in the detection process. We have chosen to utilize the Aruco environment due to its ease of use and effectiveness.

In [ ]:
import cv2
import numpy as np

aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_50)

id = 47 # This is the identifier of the bookmark, you can change it to whatever you need
img_size = 700 # Define the size of the final image
marker_img = cv2.aruco.generateImageMarker(aruco_dict, id, img_size)

cv2.imwrite('aruco{}.png'.format(id), marker_img)

### Perspective Transformation
To establish an accurate representation of the detected ROI and Thymio's environment, a perspective transformation is applied. This involves computing the transformation matrix based on the detected Aruco markers, enabling the correction of distortions and ensuring a consistent viewpoint.

We now proceed to compute the transformation. The objective is to derive the transformation matrix, denoted as M, for seamless utilization in subsequent steps, particularly within the get_ROI function.

In [ ]:
def perspective_correction(image):
    detected = cv2.aruco.detectMarkers(image, arucoDict, parameters=arucoParams)
    corners = [None]*4
    for (id, idx) in corner_ids.items():
      (center, _) = get_pos_aruco(detected, id)
      if center is not None:
        corners[idx] = center
      else:
        return None

    # Do perspective correction
    pts1 = np.array([corners[0], corners[1], corners[3], corners[2]])
    pts2 = np.float32([[res_h,res_w], [0, res_w], [res_h, 0], [0, 0]])

    return cv2.getPerspectiveTransform(pts1,pts2)


def get_ROI(img, M): 
    return cv2.warpPerspective(img,M,(res_h,res_w))

### Obstacle Detection
Having corrected the perspective, our next step involves obstacle detection within the Region of Interest (ROI). Initially, we focus on computing the positions of both the Thymio and the destination.

### Thymio and destination position finding : 
For detecting the Thymio, it is essential to compute the center, border, and angle of the Aruco marker. The computation is performed as follows: #todo -> decrire comment compute angle

In [ ]:
def find_thymio(detected):
  # Detect aruco
  (center, c) = get_pos_aruco(detected, thymio_id)

  if center is None:
    return (None, None, None)

  #compute the direction : 
  v1 = c[1] - c[0]
  v2 = c[2] - c[3]
  dir = (v1 + v2)/2
  angle = np.arctan2(dir[1], dir[0])
  
  return (c, center, angle)

In [ ]:
def find_end_point(detected):
  (center, corners) = get_pos_aruco(detected, end_point_id)
  if center is None:
    return None
  return (corners,center)

A threshold on the red channel of the image is employed to identify obstacles. The thresholding operation simplifies the detection of objects with specific color characteristics. Our approach suggests to fine tune the thresholding thanks to a slider.

In [ ]:
def process_image_red(image, red_threshold):
    red_channel = image[:, :, 0].copy()  # Create a copy of the red channel
    red_channel[red_channel > red_threshold] = 0
    _, binary = cv2.threshold(red_channel, 1, 255, cv2.THRESH_BINARY)
    return binary

The vision module, incorporating Aruco marker detection, perspective transformation, and obstacle detection, serves as a critical component in enhancing the Thymio robot's understanding of its environment. This integrated approach enables precise localization, ensuring effective navigation and interaction capabilities.

# IV. Global navigation
<a class="anchor" id="Global-navigation"></a>

**Concept**
From the Vision, the global Navigation gets an image with all the obstacles. After enlarging those, to find the paths the robot can acutally drive, a Visiblity Graph is generated and the shortest path is found using the Dijkstra Algorithm. 

<center>
<img src="ImgRapport/global_1.jpeg" alt="Map From Vision" width="320"/>
<img src="ImgRapport/global_2.png" alt="Map From Vision" width="300"/>
</center>

From the Vision, we get the left image and use OpenCV to find the edgepoints from the polygons. As sometimes there are points close to each other found by the algorithm, we use an area threshold to filter out the points that we dont want to find only the edge points of the Polygons

<center>
<img src="ImgRapport/eroded_img.png" alt="Map From Vision" width="320"/>
</center>

We then enlarge the polygons. Given that all our structures are convex shapes, we simply offset the point using vectors from the adjacent points towards the point we want to offset. If the point moves out of the map that way, we crop it off to the mapsize. We then redraw the polygons onto a black canvas, as in the picture above. This is to find out if the enlarged structures are overlapping. We apply OpenCV functions again which search for shapes, but this time we need to take in account that the resulting shapes no longer may be convex.  


In [ ]:
def offset_point(center, a, b, offset, img_shape):
    v1 = vector(a, center)
    v1 = scalarmult(v1, 1/length(v1)*offset)
    v2 = vector(b, center)
    v2 = scalarmult(v2, 1/length(v2)*offset)
    new_point = v_sum(v_sum(center, v1), v2)
    
    # Prevent offset moving point offscreen
    new_point[0] = 0 if new_point[0]<0 else new_point[0]
    new_point[0] = img_shape[1] if new_point[0]>img_shape[1] else new_point[0]
    new_point[1] = 0 if new_point[1]<0 else new_point[1]
    new_point[1] = img_shape[0] if new_point[1]>img_shape[0] else new_point[1]
    
    return new_point

When we have all edgepoints and the polygons drawn, we need to find valid connections to build the graph. To do this, we first errode the enlarged picture, to allow the found edge points to be slightly away from the structure. 


In [ ]:
def find_connections(img_enlarged, nodelist, maxx, maxy): # polygons are enlarged polygons
    poly_connections = []
    connections = []
    # Add Polygons
    _, binary_image = cv2.threshold(img_enlarged, 127, 255, cv2.THRESH_BINARY)

    # Define the kernel size for erosion
    kernel_size = 3  # You can adjust this value
    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    # Apply erosion to make structures slightly smaller
    eroded_image = cv2.erode(binary_image, kernel, iterations=6)

We then go through all point pairs, and by connecting them with a line and checking if the line intersects with the erroded polygons, we can define if a pair of points has a valid edge between them

In [ ]:
def check_line_through_white_area(image, start_point, end_point):
    # Assuming the image is a binary image from erosion
    white_mask = (image == 255).astype(np.uint8)
 
    # Create an empty mask for the line, ensuring it's uint8 type
    line_mask = np.zeros_like(white_mask, dtype=np.uint8)

    # Draw the line on the mask
    cv2.line(line_mask, start_point, end_point, 1, thickness=1)

    # Check if any line pixel is also white in the image
    intersection = cv2.bitwise_and(white_mask, line_mask)
    return np.any(intersection)


<center>
<img src="ImgRapport/global_3.png" alt="Map From Vision" width="300"/>
<img src="ImgRapport/erroded_path.png" alt="Map From Vision" width="300"/>
</center>

We then run a Dijkstra algorithm on the generated graph to find the shortest path which we want to follow. On the left, a result of an optimal path with all connections is shown. On the right, an example is shown using the erroded polygons.

# V. Local Navigation
<a class="anchor" id="Local-navigation"></a>

The local navigation runs continuously, serving as the Thymio's last line of defense against collisions with unforeseen or vision-unseen obstacles. In this phase, the priority is to prevent any collisions, disregarding the predetermined path provided by global navigation.

Obstacle avoidance is executed using a Neural Network. The inputs for the Neural Network are the proximity IR sensors arranged around the Thymio (prox in the code). The outputs consist of speed inputs for the right and left motors (motorL and motorR in the code). Between these outputs, there are weights organized in a 2x5 matrix format (NNW in the code). These weights are designed to assign more significance to obstacles directly in front of the Thymio than those on its sides. This adjustment helps the robot turn more when the obstacle is in front, as opposed to being at the extremities of the robot.

To prevent the robot from being repulsed by distant walls, a threshold is implemented. Below this threshold, the robot does not consider the obstacle. Additionally, a Gain is applied to the motor speed values, allowing for the adjustment of the avoidance system's reactivity according to preferences.

In [ ]:
import numpy as np

def LocalAvoidance(prox) : 
    
    NNW = np.array([[2, 3, -4, -3, -2],[-2, -3, -4, 3, 2]])
    threshold = 500
    Gain = 0.01
    obstacle_detected = False
    prox_for = np.zeros(5)

    for i in range(5):
        prox_for[i] = prox[i]
        if(prox[i] > threshold) :
            obstacle_detected = True

    if not(obstacle_detected) :
        return 0, 0

    elif obstacle_detected :
        Y = np.dot(NNW, prox_for) * Gain
        motor_L = Y[0] 
        motor_R = Y[1]
        return motor_L, motor_R

#Test part : 
prox_test = [2200,1500, 400, 0,0 ] # sensor values wich correspond to an right sided obstacle
print(LocalAvoidance(prox_test))   # output : we avoid the obstacle by turning on the right 

Bellow is a video of the robot avoiding an obstacle. For the demonstration, the speed motors on both side has been set to 200. 

# VI. Filtering
<a class="anchor" id="Filtering"></a>

# VII. Motion Control
<a class="anchor" id="Motion-control"></a>


The motion control's primary objective is to determine the motor speeds for the left and right sides based on the robot's position (x, y), orientation (theta), and the array of path coordinates. As a reminder, the global navigation part provides us with a list of points that the Thymio needs to reach.

For accurate line following, we have drawn inspiration from steering behavior path following. In this context, steering behaviors encompass a set of algorithms or techniques used to control the movement of autonomous agents or entities in a virtual environment, simulating natural and intelligent motion.

Within these techniques, we specifically focus on the "seek" concept, which involves moving towards a target point defined on the path. In our code, this target point is referred to as the "carrot." In summary, the path-following algorithm comprises two parts:

- Compute the carrot position.
- Move toward the carrot using a P controller.


**Compute the carrot position :**
To compute the carrot position, we inspired ourselves from this video : https://www.youtube.com/watch?v=rlZYT-uvmGQ&t=16s 

(1) Initially, we calculate the position projection, denoted as 'projection,' using the Thymio's orientation and a parameter called d_projection.

(2) Subsequently, we create two vectors. Vector A represents the vector from the beginning of the segment to the path point, and vector B is the normalized vector from the beginning of the segment to the end.

(3) Next, we perform a scalar projection of A onto B, resulting in the value sp.

(4) Finally, we compute the carrot position by projecting sp in the direction of B, starting from the robot's position.

The first parameter to adjust is d_projection, which determines the horizon of the Thymio. If it's too short, the Thymio will react at the last moment during turns. If it's too long, the Thymio will be less responsive.

The second parameter is Kpteta, which is experimentally determined. If it's too large, the Kalman filter may oscillate; if it's too low, the Thymio will turn too slowly.


**Go to the carrot :**
This section is responsible for determining the motor speeds based on inputs such as the robot's position, the angle (theta), the carrot position, and the projection.

(5) Initially, we set the motor speeds to the same value, representing the speed of the robot when the carrot is directly in front of the Thymio.

(6) Subsequently, we calculate the distance between the 'projection' and the carrot.
        If the distance is less than the margin, we keep the motor speeds unchanged.
        Otherwise, we compute the angle Phi, which is the angle between the Thymio's orientation and the ideal orientation to reach the carrot.
                Then, we adjust the motor speeds using a P controller: motorR = motorR - phi * KPangle.

Here is a schema to make it more simple and the code :

<img src="ImgRapport\Schema_pathfollow.jpg" style="width: 700px;">

In [ ]:
KPteta = 70
segment_idx = 0

# function to know the distance beetwin 2 points
def distance(point1, point2):
    
    return math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)

# function to compute a vector form 2 points
def vector_compute(point1, point2):

    vecteur = (point2[0] - point1[0], point2[1] - point1[1])
    return vecteur

# fucntion to bond an angle beetwin -pi and pi  
def adjust_angle(_angle):

    while _angle > math.pi:
        _angle -= 2*math.pi
    while _angle < -math.pi:
        _angle += 2*math.pi
    return _angle



def go_to_carrot(_position, _carrot, _teta, _margin) : 

    motorL = 0
    motorR = 0
    motorL = 70
    motorR = 70

    d = distance(_position, _carrot)    # function to know the distance beetwin 2 points
    vector_carrot = vector_compute(_position, _carrot)
    if d > _margin :
        phi =  math.atan2(vector_carrot[1],vector_carrot[0]) - _teta
        phi = adjust_angle(phi)     
        motorL = motorL + phi * KPteta
        motorR = motorR - phi * KPteta

    return motorL, motorR


def follow_path(position, teta, path, path_has_been_done) :

    # Initialization of the variables
    projection = np.array([0,0])
    carrot = np.array([0,0])
    motorL = 0
    motorR = 0
    global segment_idx  #index of the target segment. (the segment wich the robot is following)
    
    # Set parameters
    margin = 5 # margin around the line  
    d_projection = 30   # distance from the robot to the projection 
    has_finished = 0    # flag to know if the robot has reached the last point
    limit_distance = 40     # distance in pixels to know if the robot has reached the end of an segment
    

    # Ckech if the path planning just came to be done 
    if path_has_been_done == 1 :
        segment_idx = 0
        path_has_been_done = 0

    # Check if the position has reached the end segment point
    if distance(position, path[segment_idx +1]) < limit_distance : 
        segment_idx += 1
        print('let s change segemnt')
        if segment_idx == len(path)-1 :
            has_finished = 1
            print('End')

    # Check if the Thymio has reached the end 
    if has_finished == 1:
        motorL = 0
        motorR = 0
        return motorL, motorR, has_finished
    else : 
        # step(1)
        projection = position + np.array([d_projection*math.cos(teta), d_projection*math.sin(teta)])
        # step(2)
        A = vector_compute(path[segment_idx], projection)
        B = vector_compute(path[segment_idx], path[segment_idx+1])
        Bnormal = B / np.linalg.norm(B)
        #step(3)
        sp = abs(np.dot(A,Bnormal))
        maxsp = distance(path[segment_idx],path[segment_idx+1])
        if sp >maxsp : 
            sp = maxsp
        #step(4)
        carrot = path[segment_idx] + Bnormal * sp
        
        
        motorL, motorR = go_to_carrot(position, carrot, teta, Bnormal, margin)
        return motorL, motorR, has_finished




# VIII . Conclusion
<a class="anchor" id="Conclusion"></a>